<a href="https://colab.research.google.com/github/sarikasea/Mythbusters/blob/main/V3_Mythbusting_CrewAI_Agent_Article_Writer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
print(np.__version__+' is numpy version')

1.26.4 is numpy version


In [3]:
!pip install -q crewai crewai_tools langchain_community langchain_google_genai google-serp-api python-dotenv
print('done - crew ai & langchain installed')

done - crew ai & langchain installed


In [2]:
import os
import warnings
from crewai import Agent, Task, Crew
from crewai_tools import SerperDevTool
from langchain.tools import Tool
from langchain_openai import ChatOpenAI
from IPython.display import display, Markdown
from google.colab import userdata

# Ignore unnecessary warnings
warnings.filterwarnings('ignore')

/usr/local/lib/python3.11/dist-packages/pydantic/fields.py:1093: PydanticDeprecatedSince20: Using extra keyword arguments on `Field` is deprecated and will be removed. Use `json_schema_extra` instead. (Extra keys: 'required'). Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  warn(


In [4]:
import os
from crewai_tools import SerperDevTool
from getpass import getpass

os.environ["SERPER_API_KEY"] =  userdata.get('SERPER_API_KEY')
search_tool = SerperDevTool()

In [5]:
# Now, let's use the search_tool to perform a search
try:
    # Pass the search term as a keyword argument 'query'
    results = search_tool.run(query="latest news on veganism")

    # The 'results' variable is already a formatted string from the tool.
    # We can just print it directly.
    print(results)

except Exception as e:
    print(f"An error occurred: {e}")

Using Tool: Search the internet with Serper
{'searchParameters': {'q': 'latest news on veganism', 'type': 'search', 'num': 10, 'engine': 'google'}, 'organic': [{'title': "Today's Top Vegan News Stories - VegNews.com", 'link': 'https://vegnews.com/news', 'snippet': "Food News of the Week: Vegan Octopus Returns, Beyond's Whole-Cut Steak, · Chrissy Teigen's Reflux Struggles Are So Relatable, and Why We Love the Gut-Friendly.", 'position': 1}, {'title': 'Plant Based News: Home', 'link': 'https://plantbasednews.org/', 'snippet': "James Cromwell Says He Went Vegan On Day Two Of Filming 'Babe'. After nearly 20 years of vegetarianism, James Cromwell went vegan on the second day of ...", 'position': 2, 'sitelinks': [{'title': 'All News', 'link': 'https://plantbasednews.org/all/'}, {'title': 'About Us', 'link': 'https://plantbasednews.org/about-us/'}, {'title': 'Other News', 'link': 'https://plantbasednews.org/category/news/'}, {'title': 'Opinion', 'link': 'https://plantbasednews.org/category/op

In [6]:
#from google.colab import userdata

import os
from langchain_google_genai import ChatGoogleGenerativeAI

# --- Set the Environment Variable Here ---
# Paste your key from the working cell in your screenshot
os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

# Now, the rest of your code will work because the environment variable is set
model_name = os.getenv("GOOGLE_MODEL", "gemini-1.5-pro-latest")

print(f"Successfully loaded Google LLM: {model_name}")

Successfully loaded Google LLM: gemini-1.5-pro-latest


In [17]:
from langchain_google_genai import ChatGoogleGenerativeAI

# This is the most reliable way to configure the Gemini LLM for CrewAI.
# It ensures the model name is sent correctly to Google's API.
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-pro-latest",
    # The API key is automatically read from your environment variables,
    # so you don't need to pass it here.
    temperature=0.7,
    verbose=True
)

# This test call will now work correctly
try:
    response = llm.invoke("Explain how AI works in a few words")
    print(response.content)
except Exception as e:
    print(f"An error occurred: {e}")

AI mimics human intelligence using algorithms and data.


#AGENTS

In [18]:
ResearchAgent = Agent(
    name="HistoricalFactFinder",
    role=(
        "You are a specialized historical researcher. Your task is to gather detailed, "
        "fact-checked information on a given historical figure ({topic}). You must find "
        "information to cover the following specific sections: "
        "1. Era/Country, 2. Love Life, 3. Scientific/Political Contribution, "
        "4. Misunderstanding/Scandal, 5. End of Life/Legacy, "
        "6. Recommended books about them, 7. Notable Quotes from them, "
        "8. Songs/Movies about them."
    ),
    goal=(
        "Compile a comprehensive report on {topic} with detailed notes and verifiable "
        "sources for each of the eight required sections. This report will be used to "
        "write a structured, myth-busting article."
    ),
    backstory=(
        "You are a meticulous historian with a passion for primary sources and "
        "academic databases. You excel at separating fact from fiction and providing "
        "the raw material for compelling historical narratives."
    ),
    tools=[search_tool], # Add other tools like web_search_tool as needed
    llm=llm,
    memory=True,
    allow_delegation=False,
    verbose=True,
)

# Writer agent: Now tasked with writing an article with the specified structure.
WriterAgent = Agent(
    name="NarrativeWeaver",
    role=(
        "You are an expert writer specializing in historical biographies. Your task is to "
        "transform research notes about {topic} into an engaging myth-busting article."
    ),
    goal=(
        "Write a captivating article about {topic} that is clearly structured into the "
        "following eight sections: 1. Historical Context (Era/Country), "
        "2. Personal Life & Relationships, 3. Major Contributions & Achievements, "
        "4. Scandals & Misconceptions, 5. Final Years & Lasting Legacy, "
        "6. Further Reading (Recommended Books), 7. Memorable Quotes, "
        "8. Pop Culture Portrayals (Songs/Movies). The article must be engaging, "
        "educational, and based strictly on the provided research."
    ),
    backstory=(
        "As a journalist and cultural analyst for a popular history blog, you know how to "
        "make history come alive. You craft narratives that are not only informative but "
        "also challenge common misinformation, especially concerning women in history."
    ),
    llm=llm,
    memory=True,
    allow_delegation=False,
    verbose=True,
)

# Editor agent: Now responsible for checking for the presence and quality of the 8 sections.
EditorAgent = Agent(
    name="ContentChief",
    role=(
        "You are a meticulous editor with expertise in feminist history and public education."
        "You will review a draft article for clarity, factual accuracy, and tone."
    ),
    goal=(
        "Polish the myth-busting article about {topic}, ensuring it aligns with the goal "
        "of accessible public education. You must verify that: "
        "1. All facts are clearly explained and well-cited. "
        "2. The tone is professional yet accessible. "
        "3. All eight required sections (Era, Love Life, Contributions, Scandal, Legacy, "
        "Books, Quotes, Media) are present, well-developed, and seamlessly integrated."
    ),
    backstory=(
        "With years of experience editing for historical journals and online magazines, "
        "you have a sharp eye for detail and a deep understanding of how to present "
        "complex historical truths to a broad audience. Your standard is excellence."
    ),
    llm=llm,
    memory=True,
    allow_delegation=False,
    verbose=True,
)

In [19]:
def run_myth_busting_workflow(topic):
    research_task = Task(
        agent=ResearchAgent,
        description=f"Research common myths about {topic} and gather accurate historical information to refute them.",
        expected_output="Bullet points with sources about myths and true facts"
    )

#TASKS


In [20]:
from crewai import Agent, Task, Crew

# (You would have the Agent definitions and the 'article_sections' dictionary from the previous step here)
# ...

# ==============================================================================
# 3. DEFINE THE TASKS THAT ALIGN WITH THE AGENT GOALS
# ==============================================================================

# Note: The `description` for each task is now precisely aligned with the agent's role
# and our shared `article_sections` structure.

research_task = Task(
    agent=ResearchAgent,
    description=(
        "Find detailed, fact-checked information on the historical figure: {topic}. "
        "Your research must provide material for all eight of the required sections. "
        "Focus on uncovering details that can bust common myths or add surprising depth."
    ),
    expected_output=(
        "A comprehensive report as a single text document. The report must be clearly "
        "structured with headings for each of the 8 required sections, containing detailed "
        "notes and verifiable sources for each section."
    )
)

writing_task = Task(
    agent=WriterAgent,
    description=(
        "Using the provided research report on {topic}, write a captivating, myth-busting "
        "article of approximately 1200 words. The article's structure must strictly "
        "follow the 8 required sections. Weave the research into a compelling narrative "
        "that is both educational and engaging."
    ),
    expected_output=(
        "A complete, well-written article draft, formatted with clear headings for each of the "
        "8 sections. The tone should be engaging and accessible to a general audience."
    ),
    context=[research_task] # This task depends on the output of the research_task
)

editing_task = Task(
    agent=EditorAgent,
    description=(
        "Review the provided article draft about {topic}. Your review must focus on several key areas: "
        "1. Verify that all 8 required sections are present, well-developed, and logically structured. "
        "2. Check all facts against the source material for accuracy. "
        "3. Polish the writing for clarity, engagement, and consistent tone. "
        "4. Ensure the article successfully fulfills its 'myth-busting' goal."
    ),
    expected_output=(
        "The final, publishable version of the article on {topic}. It should be free of errors, "
        "factually pristine, and ready for an audience."
    ),
    context=[writing_task] # This task depends on the output of the writing_task
)



In [22]:
# Define the list of topics you want to process
topics_to_process = ["Cleopatra", "Marie Curie"]

# Loop through each topic and run the full workflow for each one
for topic in topics_to_process:
    print(f"\n\n=============================================")
    print(f"🚀 Starting New Workflow for Topic: {topic}")
    print(f"=============================================")

    # Create the Crew for the current topic
    crew = Crew(
        agents=[ResearchAgent, WriterAgent, EditorAgent],
        tasks=[research_task, writing_task, editing_task],
        verbose=True
    )

    # Kick off the workflow with the current topic
    result = crew.kickoff(inputs={"topic": topic})

    # Print the final result for the current topic
    print(f"\n\n✅ Final Result for {topic}:")
    display(Markdown(result))



🚀 Starting New Workflow for Topic: Cleopatra


╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: f01565be-95d1-4020-a230-4b7f288ae3b4                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: You are a specialized historical researcher. Your task is to gather detailed, fact-checked information  │
│  on a given historical figure (Cleopatra). You must find information to cover the following specific sections:  │
│  1. Era/Country, 2. Love Life, 3. Scientific/Political Contribution, 4. Misunderstanding/Scandal, 5. End of     │
│  Life/Legacy, 6. Recommended books about them, 7. Notable Quotes from them, 8. Songs/Movies about them.         │
│                                                                                                                 │
│  Task: Find detailed, fact-checked information on the historical figure: Cleopatra. Your research must provide  │
│  material for all eight of the required sections. Focus on uncovering details that can bust common myths or     │
│  add surprising depth.                                                                                          │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭─────────────────────────────────────────────────── LLM Error ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  ❌ LLM Call Failed                                                                                             │
│  Error: litellm.BadRequestError: LLM Provider NOT provided. Pass in the LLM provider you are trying to call.    │
│  You passed model=models/gemini-1.5-pro-latest                                                                  │
│   Pass model as E.g. For 'Huggingface' inference endpoints pass in                                              │
│  `completion(model='huggingface/starcoder',..)` Learn more: https://docs.litellm.ai/docs/providers              │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────── Task Failure ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Failed                                                                                                    │
│  Name: 16bae635-daa7-4479-ba1b-055d7cc36759                                                                     │
│  Agent: You are a specialized historical researcher. Your task is to gather detailed, fact-checked information  │
│  on a given historical figure (Cleopatra). You must find information to cover the following specific sections:  │
│  1. Era/Country, 2. Love Life, 3. Scientific/Political Contribution, 4. Misunderstanding/Scandal, 5. End of     │
│  Life/Legacy, 6. Recommended books about them, 7. Notable Quotes from them, 8. Songs/Movies about them.         │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────── Crew Failure ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Failed                                                                                          │
│  Name: crew                                                                                                     │
│  ID: f01565be-95d1-4020-a230-4b7f288ae3b4                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output:                                                                                                  │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

BadRequestError: litellm.BadRequestError: LLM Provider NOT provided. Pass in the LLM provider you are trying to call. You passed model=models/gemini-1.5-pro-latest
 Pass model as E.g. For 'Huggingface' inference endpoints pass in `completion(model='huggingface/starcoder',..)` Learn more: https://docs.litellm.ai/docs/providers